<a href="https://colab.research.google.com/github/Kh-Harakeh/IOL-SAT-Solver/blob/main/sat_linearsolver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def T(formula):
    def negate(expression):
        return '¬' + expression

    def process_operation(expression, operator, operation_func):
        parts = expression.split(operator)
        left = transform(parts[0].strip())
        right = transform(parts[1].strip())
        return operation_func(left, right)

    def conjunction(left, right):
        return left + ' ∧ ' + right

    def disjunction(left, right):
        return '¬(¬' + left + ' ∧ ¬' + right + ')'

    def implication(left, right):
        return '¬(' + left + ' ∧ ¬' + right + ')'

    def transform(expression):
        if expression.isalpha():
            return expression

        while expression.startswith('(') and expression.endswith(')'):
            expression = expression[1:-1]

        if expression.startswith('¬'):
            inner = transform(expression[1:])
            return negate(inner)

        for op, func in [('∧', conjunction), ('∨', disjunction), ('→', implication)]:
            if op in expression:
                return process_operation(expression, op, func)

        return expression

    return transform(formula)

class ExpressionNode:
    def __init__(self, content, left=None, right=None):
        self.content = content
        self.left = left
        self.right = right

def parse_expression(expression):
    def remove_outer_parentheses(expression):
        while expression.startswith('(') and expression.endswith(')'):
            expression = expression[1:-1].strip()
        return expression

    def find_main_conjunction(expression):
        depth = 0
        for i, char in enumerate(expression):
            if char == '(':
                depth += 1
            elif char == ')':
                depth -= 1
            elif depth == 0 and char == '∧':
                return i
        return -1

    expression = remove_outer_parentheses(expression)
    conj_index = find_main_conjunction(expression)

    if conj_index != -1:
        return ExpressionNode('∧',
                              parse_expression(expression[:conj_index].strip()),
                              parse_expression(expression[conj_index+1:].strip()))
    elif expression.startswith('¬'):
        return ExpressionNode('¬', right=parse_expression(expression[1:].strip()))
    else:
        return ExpressionNode(expression)

def transform_to_dag(node, memo=None):
    if node is None:
        return None

    if memo is None:
        memo = {}

    left_dag = transform_to_dag(node.left, memo)
    right_dag = transform_to_dag(node.right, memo)
    signature = (node.content, left_dag, right_dag)

    if signature in memo:
        return memo[signature]

    node.left = left_dag
    node.right = right_dag
    memo[signature] = node

    return node

def apply_truth_values(node, truth_values, processed):
    if node is None or node in processed:
        return True

    processed.add(node)

    if node.content == '¬':
        negated_node = node.right
        if negated_node not in truth_values:
            truth_values[negated_node] = not truth_values.get(node, True)
        elif truth_values[negated_node] == truth_values.get(node, True):
            return False

    elif node.content == '∧':
        left, right = node.left, node.right
        if truth_values.get(node, True):
            truth_values[left] = truth_values[right] = True
        else:
            if left in truth_values and right in truth_values and truth_values[left] and truth_values[right]:
                return False

    return apply_truth_values(node.left, truth_values, processed) and apply_truth_values(node.right, truth_values, processed)

def is_satisfiable(root):
    truth_values = {root: True}
    processed = set()

    if not apply_truth_values(root, truth_values, processed):
        return False

    return True

def print_tree(node, prefix="", branch='root'):
    if node is not None:
        branch_symbols = {"root": "", "left": "├── ", "right": "└── "}
        connectors = {"root": "    ", "left": "│   ", "right": "    "}

        print(prefix + branch_symbols[branch] + node.content)
        prefix += connectors[branch]

        if node.left and node.right:
            print_tree(node.left, prefix, 'left')
            print_tree(node.right, prefix, 'right')
        elif node.left:
            print_tree(node.left, prefix, 'right')
        elif node.right:
            print_tree(node.right, prefix, 'right')

def print_dag(node, prefix="", branch='root', visited=None, labels=None):
    if node is None:
        return

    if visited is None:
        visited = set()
        labels = {}

    if node not in labels:
        labels[node] = len(labels) + 1

    label = labels[node]
    branch_symbols = {"root": "", "left": "├── ", "right": "└── "}
    connectors = {"root": "    ", "left": "│   ", "right": "    "}
    shared = ' (shared)' if node in visited else ''

    print(prefix + branch_symbols[branch] + f"Node {label}: {node.content}{shared}")

    if node in visited:
        return

    visited.add(node)
    prefix += connectors[branch]

    if node.left and node.right:
        print_dag(node.left, prefix, 'left', visited, labels)
        print_dag(node.right, prefix, 'right', visited, labels)
    elif node.left:
        print_dag(node.left, prefix, 'right', visited, labels)
    elif node.right:
        print_dag(node.right, prefix, 'right', visited, labels)

In [ ]:
input_formula = "p ∧ ¬(q ∨ ¬p)"
translated_formula = T(input_formula)
print("Original formula:", input_formula)
print("Translated formula:", translated_formula)

Original formula: p ∧ ¬(q ∨ ¬p)
Translated formula: p ∧ ¬¬(¬q ∧ ¬¬p)


In [ ]:
input_formula = translated_formula
root = parse_expression(input_formula)
print("Parse tree of the formula:")
print_tree(root)

Parse tree of the formula:
∧
    ├── p
    └── ¬
        └── ¬
            └── ∧
                ├── ¬
                │   └── q
                └── ¬
                    └── ¬
                        └── p


In [ ]:
seen_subtrees = {}
root = transform_to_dag(root, seen_subtrees)
print("DAG of the formula:")
print_dag(root)

DAG of the formula:
Node 1: ∧
    ├── Node 2: p
    └── Node 3: ¬
        └── Node 4: ¬
            └── Node 5: ∧
                ├── Node 6: ¬
                │   └── Node 7: q
                └── Node 8: ¬
                    └── Node 9: ¬
                        └── Node 2: p (shared)


In [ ]:
satisfiable = is_satisfiable(root)
print("The formula is satisfiable." if satisfiable else "The formula is not satisfiable.")

The formula is satisfiable.
